In [ ]:
param_window_size = 24
param_seq_length = 287
param_num_epoch = 100
param_lstm_units = 128
param_lstm_stack = 1

num_sample = param_seq_length - param_window_size

In [ ]:
import numpy as np
import os
import pandas
import theano

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib inline

In [ ]:
dataset_file_path = './Data_RNN/201601.txt'
df = pandas.read_csv(dataset_file_path, delim_whitespace=True, header=None, usecols=[6])

# dataframe to dataset
dataset = df.values
dataset = dataset.astype('float32')

plt.plot(dataset)
plt.show()

In [ ]:


dataset = np.log10(dataset) # stationary signal
#dataset = np.diff(dataset, n=1, axis=0) # detrended signa

# dataset = np.cos(np.arange(288)*(20*np.pi/1000))[:,None]

plt.plot(dataset)   
plt.show()

In [ ]:
dataset = np.diff(dataset, n=1, axis=0) # detrended signa

# dataset = np.cos(np.arange(288)*(20*np.pi/1000))[:,None]

plt.plot(dataset)   
plt.show()

In [ ]:
dataset.shape

In [ ]:
# normalize
#scaler = MinMaxScaler(feature_range=(0,1))
#dataset = scaler.fit_transform(dataset)

In [ ]:


plt.plot(dataset)   
plt.show()

In [ ]:



dataset.shape

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        dataX.append(dataset[i:(i+look_back), 0])
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
trainX, trainY = create_dataset(dataset, param_window_size)
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))

testX, testY = create_dataset(dataset, param_window_size)
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
trainX.shape

In [ ]:
%%time

theano.config.compute_test_value = "ignore"

model = Sequential()
for i in range(param_lstm_stack):
    model.add(LSTM(param_lstm_units, batch_input_shape=(1, param_window_size, 1), stateful=True, return_sequences=True))
    model.add(Dropout(0.3))
model.add(LSTM(param_lstm_units, batch_input_shape=(1, param_window_size, 1), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
for epoch_idx in range(param_num_epoch):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False)
    model.reset_states()

In [ ]:
testScore = model.evaluate(testX, testY, batch_size=1, verbose=0)
print('Test Score: ', testScore)

model.reset_states()

In [ ]:
testX[1][:]
testY[0]

testPredict = [np.vstack([testX[0][:]])]

In [ ]:
testPredict[-1]

In [ ]:
a = np.vstack([testPredict[-1][1:],0.66])

In [ ]:
testPredict[-1][1:]

In [ ]:
test_seq_size = param_seq_length - param_window_size
curr_idx = 30

predictions = np.zeros((test_seq_size, 1))
testPredict = [np.vstack([testX[0][:]])]

look_hist = curr_idx
look_ahead = test_seq_size - curr_idx

for i in range(look_hist):
    prediction = model.predict(np.array([testPredict[-1]]), batch_size=1)
    predictions[i] = testY[i]
    testPredict.append(np.vstack([testX[i+1][:]]))

for i in range(look_ahead):    
    prediction = model.predict(np.array([testPredict[-1]]), batch_size=1)
    predictions[i + look_hist] = prediction
    testPredict.append(np.vstack([testPredict[-1][1:],prediction]))

model.reset_states()

In [ ]:
view = np.cumsum(predictions[::-1])[::1] 
view = np.reshape(view, (287,1))

plt.plot(view)
plt.show()

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
ax = plt.axes(xlim=(0, 287), ylim=(0, 1))

plotcols = ["blue","red"]

lines = []

test_seq_size = param_seq_length - param_window_size
    
for i in range(2):
    lobj = ax.plot([],[],lw=1,color=plotcols[i])[0]
    lines.append(lobj)

def init():
    for line in lines:
        line.set_data([],[])
    return lines

def animate(curr_idx):

    predictions = np.zeros((test_seq_size, 1))
    testPredict = [np.vstack([testX[0][:]])]

    look_hist = curr_idx
    look_ahead = test_seq_size - curr_idx

    for i in range(look_hist):
        prediction = model.predict(np.array([testPredict[-1]]), batch_size=1)
        predictions[i] = testY[i]
        testPredict.append(np.vstack([testX[i+1][:]]))
        
    for i in range(look_ahead):    
        prediction = model.predict(np.array([testPredict[-1]]), batch_size=1)
        predictions[i + look_hist] = prediction
        testPredict.append(np.vstack([testPredict[-1][1:],prediction]))
    
    model.reset_states()
    
    x = np.linspace(0, test_seq_size-1, test_seq_size)
    y = [testY, predictions]
    
    #for index in range(0,1):
    for i, line in enumerate(lines):
        line.set_data(x, y[i]) # set data for each line separately. 

    return lines

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=test_seq_size, interval=20, blit=True)

result_filename = 'w' + str(param_window_size) + '_' + 'u' + str(param_lstm_units) + '_' + 's' + str(param_lstm_stack) + '_' + 'e' + str(param_num_epoch)

anim.save( result_filename + '.gif', writer='imagemagick', fps=30, dpi=100)

plt.show()